In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,mean,max,min,count, when
spark=SparkSession.builder.appName("Titanic Data Analysis").getOrCreate()
from google.colab import files
uploaded=files.upload()

Saving Titanic-Dataset.csv to Titanic-Dataset (1).csv


In [ ]:
from os import write
data_path=list(uploaded.keys())[0]
df=spark.read.csv(data_path,header=True,inferSchema=True)
df.show(5)


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [ ]:
df.printSchema()
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")


root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

Number of rows: 891
Number of columns: 12


In [ ]:
df.select([(count(when(col(c).isNull(),c))/df.count()).alias(c) for c in df.columns]).show()
df=df.drop("Cabin")
mean_age=df.select(mean(col("Age"))).collect()[0][0]
df=df.fillna({"Age":mean_age})
df=df.dropna()
df.groupBy("Survived").count().show()
df.groupby("Survived").agg(mean("Age")).show()
df.groupby("Sex","Survived").count().show()
df.groupby("Pclass","Survived").count().show()
df=df.withColumn("FamilySize",col("SibSp")+col("Parch")+1)
df.groupBy("FamilySize","Survived").count().show()
df.write.csv("cleaned_titanic.csv", header=True, mode="overwrite") # Setting mode to 'overwrite'
spark.stop()

+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Embarked|FamilySize|
+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+
|        0.0|     0.0|   0.0| 0.0|0.0|0.0|  0.0|  0.0|   0.0| 0.0|     0.0|       0.0|
+-----------+--------+------+----+---+---+-----+-----+------+----+--------+----------+

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  340|
|       0|  549|
+--------+-----+

+--------+------------------+
|Survived|          avg(Age)|
+--------+------------------+
|       1|28.423600346020766|
|       0|30.415099646415896|
+--------+------------------+

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  231|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|     1|       